In [1]:
import numpy as np # 
import pandas as pd 
from sklearn.preprocessing import PolynomialFeatures
import time
import datetime as dt
import re
import lightgbm as lgb 
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import os
import json
import operator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

# 特征工程
## 基本信息表

In [2]:
'''
step1 读入数据
'''
profile_test = pd.read_csv('test/test_profile.csv')
profile_train = pd.read_csv('train/train_profile.csv')
#user=user.drop_duplicates()

In [3]:
print(len(profile_test ),len(profile_train ))

16109 64436


In [4]:
dataset=profile_train

label=dataset[['用户标识']]
dataset=dataset.drop(['用户标识'],axis=1)
dataset=pd.get_dummies(dataset,columns=['性别','职业','教育程度','婚姻状态','户口类型'])


#多项式处理
poly = PolynomialFeatures(degree=2,interaction_only=True)
dataset=poly.fit_transform(dataset)

df = pd.DataFrame(dataset.reshape(64436,301))
df.interpolate(axis=1).values.reshape(dataset.shape)

label = label.reset_index(drop= True)
df = df.reset_index(drop= True)
user_feature=pd.concat([label,df],axis=1)

profile_train_feature = user_feature

In [5]:

dataset=profile_test

label=dataset[['用户标识']]
dataset=dataset.drop(['用户标识'],axis=1)
dataset=dataset.drop(['signAB'],axis=1)
dataset=pd.get_dummies(dataset,columns=['性别','职业','教育程度','婚姻状态','户口类型'])

#多项式处理
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2,interaction_only=True)
dataset=poly.fit_transform(dataset)

df = pd.DataFrame(dataset.reshape(16109,301))
df.interpolate(axis=1).values.reshape(dataset.shape)

label = label.reset_index(drop= True)
df = df.reset_index(drop= True)
user_feature=pd.concat([label,df],axis=1)

profile_test_feature = user_feature

##  银行流水表

In [6]:
'''
step1 读取数据
'''
bankStatement_test = pd.read_csv('test/test_bankStatement.csv')
bankStatement_train = pd.read_csv('train/train_bankStatement.csv')


In [7]:
'''
step2 特征提取
'''
def get_bank_feature(bank):

    银行流水记录=bank
    #将时间戳转换为真实日期形式提取月份
    银行流水记录['流水日期'] = 银行流水记录['流水时间'].apply(lambda x :time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(x)))
    银行流水记录['流水日期']=pd.to_datetime(银行流水记录['流水日期'])
    银行流水记录['month'] = 银行流水记录['流水日期'].apply(lambda x :x.strftime('%m'))

    #划分  收入 支出 工资
    银行流水记录_收入=银行流水记录[银行流水记录["交易类型"]==0][["用户标识","流水时间","交易金额",'month']]
    银行流水记录_支出=银行流水记录[银行流水记录["交易类型"]==1][["用户标识","流水时间","交易金额",'month']]
    银行流水记录_工资=银行流水记录[银行流水记录["工资收入标记"]==1][["用户标识","流水时间","交易金额",'month']]

    银行流水记录_收入=银行流水记录_收入.rename(columns={'交易金额':'收入金额'})
    银行流水记录_支出=银行流水记录_支出.rename(columns={'交易金额':'支出金额'})
    银行流水记录_工资=银行流水记录_工资.rename(columns={'交易金额':'工资金额'})

#-------------------总体特征提取----------
    gb1=银行流水记录_收入.groupby(["用户标识"],as_index=False)#收入统计
    gb2=银行流水记录_支出.groupby(["用户标识"],as_index=False)#支出统计
    gb3=银行流水记录_工资.groupby(["用户标识"],as_index=False)#工资收入统计
    x1=gb1['收入金额'].agg({'用户收入笔数' : 'count','用户收入总计':'sum','用户收入均值':'mean','用户收入标准差':'std'})
    x2=gb2['支出金额'].agg({'用户支出笔数' : 'count','用户支出总计':'sum','用户支出均值':'mean','用户支出标准差':'std'})
    x3=gb3['工资金额'].agg({'用户工资笔数' : 'count','用户工资总计':'sum','用户工资均值':'mean','用户工资标准差':'std'})

    print(len(x1),len(x2),len(x3))
    feature=pd.merge(x1, x2,how='outer', on = "用户标识")
    feature=pd.merge(feature, x3,how='outer', on = "用户标识")
    
    
#———————————————收入 month维度下的特征提取---------------------
    d=银行流水记录_收入

    #-----month维度下的特征提取
    #sum
    收入金额_sum =d['收入金额'].groupby([d['用户标识'],d['month']]).sum()
    收入金额_sum=pd.DataFrame(收入金额_sum.unstack())

    name_dic={}
    for i in 收入金额_sum.columns:
        name_dic[str(i)]='收入金额_sum_month_'+str(i)
    收入金额_sum=收入金额_sum.rename(columns=name_dic)

    #mean
    收入金额_means =d['收入金额'].groupby([d['用户标识'],d['month']]).mean()
    收入金额_means=pd.DataFrame(收入金额_means.unstack())

    name_dic={}
    for i in 收入金额_means.columns:
        name_dic[str(i)]='收入金额_mean_month_'+str(i)
    收入金额_means=收入金额_means.rename(columns=name_dic)

    #std
    收入金额_std =d['收入金额'].groupby([d['用户标识'],d['month']]).std()
    收入金额_std=pd.DataFrame(收入金额_std.unstack())

    name_dic={}
    for i in 收入金额_std.columns:
        name_dic[str(i)]='收入金额_std_month_'+str(i)
    收入金额_std=收入金额_std.rename(columns=name_dic)
    #max
    收入金额_max =d['收入金额'].groupby([d['用户标识'],d['month']]).max()
    收入金额_max=pd.DataFrame(收入金额_max.unstack())

    name_dic={}
    for i in 收入金额_max.columns:
        name_dic[str(i)]='收入金额_max_month_'+str(i)
    收入金额_max=收入金额_max.rename(columns=name_dic)
    #min
    收入金额_min =d['收入金额'].groupby([d['用户标识'],d['month']]).min()
    收入金额_min=pd.DataFrame(收入金额_min.unstack())

    name_dic={}
    for i in 收入金额_min.columns:
        name_dic[str(i)]='收入金额_min_month_'+str(i)
    收入金额_min=收入金额_min.rename(columns=name_dic)

    银行流水收入_month=pd.merge(收入金额_sum,收入金额_means,how="inner",on="用户标识")
    银行流水收入_month=pd.merge(银行流水收入_month,收入金额_max,how="inner",on="用户标识")
    银行流水收入_month=pd.merge(银行流水收入_month,收入金额_min,how="inner",on="用户标识")
    银行流水收入_month=pd.merge(银行流水收入_month,收入金额_std,how="inner",on="用户标识")

    #------------------------------支出  month维度下的特征提取---------------------------
    d=银行流水记录_支出

    #sum
    支出金额_sum =d['支出金额'].groupby([d['用户标识'],d['month']]).sum()
    支出金额_sum=pd.DataFrame(支出金额_sum.unstack())

    name_dic={}
    for i in 支出金额_sum.columns:
        name_dic[str(i)]='支出金额_sum_month_'+str(i)
    支出金额_sum=支出金额_sum.rename(columns=name_dic)

    #mean
    支出金额_mean =d['支出金额'].groupby([d['用户标识'],d['month']]).mean()
    支出金额_means=pd.DataFrame(支出金额_mean.unstack())

    name_dic={}
    for i in 支出金额_means.columns:
        name_dic[str(i)]='支出金额_mean_month_'+str(i)
    支出金额_means=支出金额_means.rename(columns=name_dic)

    #std
    支出金额_std =d['支出金额'].groupby([d['用户标识'],d['month']]).std()
    支出金额_std=pd.DataFrame(支出金额_std.unstack())

    name_dic={}
    for i in 支出金额_std.columns:
        name_dic[str(i)]='支出金额_std_month_'+str(i)
    支出金额_std=支出金额_std.rename(columns=name_dic)
    #max
    支出金额_max =d['支出金额'].groupby([d['用户标识'],d['month']]).max()
    支出金额_max=pd.DataFrame(支出金额_max.unstack())

    name_dic={}
    for i in 支出金额_max.columns:
        name_dic[str(i)]='支出金额_max_month_'+str(i)
    支出金额_max=支出金额_max.rename(columns=name_dic)
    #min
    支出金额_min =d['支出金额'].groupby([d['用户标识'],d['month']]).min()
    支出金额_min=pd.DataFrame(支出金额_min.unstack())

    name_dic={}
    for i in 支出金额_min.columns:
        name_dic[str(i)]='支出金额_min_month_'+str(i)
    支出金额_min=支出金额_min.rename(columns=name_dic)

    银行流水支出_month=pd.merge(支出金额_sum,支出金额_means,how="inner",on="用户标识")
    银行流水支出_month=pd.merge(银行流水支出_month,支出金额_max,how="inner",on="用户标识")
    银行流水支出_month=pd.merge(银行流水支出_month,支出金额_min,how="inner",on="用户标识")
    银行流水支出_month=pd.merge(银行流水支出_month,支出金额_std,how="inner",on="用户标识")

    #------------------------------工资 month维度下的特征提取------------------------------------
    d=银行流水记录_工资

    #sum
    工资金额_sum =d['工资金额'].groupby([d['用户标识'],d['month']]).sum()
    工资金额_sum=pd.DataFrame(工资金额_sum.unstack())

    name_dic={}
    for i in 工资金额_sum.columns:
        name_dic[str(i)]='工资金额_sum_month_'+str(i)
    工资金额_sum=工资金额_sum.rename(columns=name_dic)

    #mean
    工资金额_mean =d['工资金额'].groupby([d['用户标识'],d['month']]).mean()
    工资金额_mean=pd.DataFrame(工资金额_mean.unstack())

    name_dic={}
    for i in 工资金额_mean.columns:
        name_dic[str(i)]='工资金额_mean_month_'+str(i)
    工资金额_means=工资金额_mean.rename(columns=name_dic)

    #std
    工资金额_std =d['工资金额'].groupby([d['用户标识'],d['month']]).std()
    工资金额_std=pd.DataFrame(工资金额_std.unstack())

    name_dic={}
    for i in 工资金额_std.columns:
        name_dic[str(i)]='工资金额_std_month_'+str(i)
    工资金额_std=工资金额_std.rename(columns=name_dic)
    #max
    工资金额_max =d['工资金额'].groupby([d['用户标识'],d['month']]).max()
    工资金额_max=pd.DataFrame(工资金额_max.unstack())

    name_dic={}
    for i in 工资金额_max.columns:
        name_dic[str(i)]='工资金额_max_month_'+str(i)
    工资金额_max=工资金额_max.rename(columns=name_dic)
    #min
    工资金额_min =d['工资金额'].groupby([d['用户标识'],d['month']]).min()
    工资金额_min=pd.DataFrame(工资金额_min.unstack())

    name_dic={}
    for i in 工资金额_min.columns:
        name_dic[str(i)]='工资金额_min_month_'+str(i)
    工资金额_min=工资金额_min.rename(columns=name_dic)

    银行流水工资_month=pd.merge(工资金额_sum,工资金额_means,how="inner",on="用户标识")
    银行流水工资_month=pd.merge(银行流水工资_month,工资金额_max,how="inner",on="用户标识")
    银行流水工资_month=pd.merge(银行流水工资_month,工资金额_min,how="inner",on="用户标识")
    银行流水工资_month=pd.merge(银行流水工资_month,工资金额_std,how="inner",on="用户标识")
    
    # 特征合并
    bank_feature=pd.merge(feature,银行流水收入_month,how='outer',on='用户标识')
    bank_feature=pd.merge(bank_feature,银行流水支出_month,how="outer",on="用户标识")
    bank_feature=pd.merge(bank_feature,银行流水工资_month,how="outer",on="用户标识")
    
    return (bank_feature)

In [8]:
bank_test_feature = get_bank_feature(bankStatement_test)
bank_train_feature = get_bank_feature(bankStatement_train)

2182 2515 947
7989 9588 3079


##  信用卡账单记录

In [9]:
'''
step1 读取数据  
'''
credit_test = pd.read_csv('test/test_creditBill.csv')
credit_train = pd.read_csv('train/train_creditBill.csv')

In [10]:
'''
    step 2  特征提取
'''
def get_credit_feature(bill):
    d=bill

    ###补充特征，增加整体的各种统计信息sum count max min mean std var等
    gb=d.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','还款状态']].groupby(["用户标识"],as_index=False)
    整体账单sum=gb.sum()
    整体账单sum.columns = ['用户标识', '整体上期账单金额sum', '整体上期还款金额sum','整体信用卡额度sum','整体本期账单余额sum','整体还款状态sum']
    feature=整体账单sum
    feature['整体上期还款金额sum与整体上期账单金额sum差值']=feature['整体上期还款金额sum']-feature['整体上期账单金额sum']
    feature['整体信用卡额度sum与整体本期账单余额sum差值']=feature['整体信用卡额度sum']-feature['整体本期账单余额sum']

    整体账单max=gb.max()
    整体账单max.columns = ['用户标识', '整体上期账单金额max', '整体上期还款金额max','整体信用卡额度max','整体本期账单余额max','整体还款状态max']
    feature=pd.merge(feature, 整体账单max,how='left', on = "用户标识")
    feature['整体上期还款金额max与整体上期账单金额max差值']=feature['整体上期还款金额max']-feature['整体上期账单金额max']
    feature['整体信用卡额度max与整体本期账单余额max差值']=feature['整体信用卡额度max']-feature['整体本期账单余额max']

    整体账单min=gb.min()
    整体账单min.columns = ['用户标识', '整体上期账单金额min', '整体上期还款金额min','整体信用卡额度min','整体本期账单余额min','整体还款状态min']
    feature=pd.merge(feature, 整体账单min,how='left', on = "用户标识")
    feature['整体上期还款金额min与整体上期账单金额min差值']=feature['整体上期还款金额min']-feature['整体上期账单金额min']
    feature['整体信用卡额度min与整体本期账单余额min差值']=feature['整体信用卡额度min']-feature['整体本期账单余额min']

    整体账单mean=gb.mean()
    整体账单mean.columns = ['用户标识', '整体上期账单金额mean', '整体上期还款金额mean','整体信用卡额度mean','整体本期账单余额mean','整体还款状态mean']
    feature=pd.merge(feature, 整体账单mean,how='left', on = "用户标识")
    feature['整体上期还款金额mean与整体上期账单金额mean差值']=feature['整体上期还款金额mean']-feature['整体上期账单金额mean']
    feature['整体信用卡额度mean与整体本期账单余额mean差值']=feature['整体信用卡额度mean']-feature['整体本期账单余额mean']

    整体账单median=gb.median()
    整体账单median.columns = ['用户标识', '整体上期账单金额median', '整体上期还款金额median','整体信用卡额度median','整体本期账单余额median','整体还款状态median']
    feature=pd.merge(feature, 整体账单median,how='left', on = "用户标识")
    feature['整体上期还款金额median与整体上期账单金额median差值']=feature['整体上期还款金额median']-feature['整体上期账单金额median']
    feature['整体信用卡额度median与整体本期账单余额median差值']=feature['整体信用卡额度median']-feature['整体本期账单余额median']

    整体账单std=gb.std()
    整体账单std.columns = ['用户标识', '整体上期账单金额std', '整体上期还款金额std','整体信用卡额度std','整体本期账单余额std','整体还款状态std']
    feature=pd.merge(feature, 整体账单std,how='left', on = "用户标识")
    feature['整体上期还款金额std与整体上期账单金额std差值']=feature['整体上期还款金额std']-feature['整体上期账单金额std']
    feature['整体信用卡额度std与整体本期账单余额std差值']=feature['整体信用卡额度std']-feature['整体本期账单余额std']

    整体账单var=gb.var()
    整体账单var.columns = ['用户标识', '整体上期账单金额var', '整体上期还款金额var','整体信用卡额度var','整体本期账单余额var','整体还款状态var']
    feature=pd.merge(feature, 整体账单var,how='left', on = "用户标识")
    feature['整体上期还款金额var与整体上期账单金额var差值']=feature['整体上期还款金额var']-feature['整体上期账单金额var']
    feature['整体信用卡额度var与整体本期账单余额var差值']=feature['整体信用卡额度var']-feature['整体本期账单余额var']

    d=bill
    # 爆卡 指的是本期账单余额大于信用卡额度
    gb=d[(d['信用卡额度']<d['本期账单余额'])].groupby(["用户标识"],as_index=False)
    x1=gb['账单时间戳'].apply(lambda x:np.unique(x).size)
    x=gb['账单时间戳'].agg({'爆卡次数' : 'count'})
    x['爆卡次数(去重)']=x1
    feature=pd.merge(feature, x,how='left', on = "用户标识")

    #用户持卡数
    gb=d.groupby(["用户标识"],as_index=False)
    x=gb['银行标识'].apply(lambda x:np.unique(x).size)
    x1=gb['银行标识'].agg({'用户银行卡账单计数' : 'count'})
    x1['用户持卡数']=x
    feature=pd.merge(feature,x1,how='left', on = "用户标识")

    #去重银行卡标识后 信用卡额度与可用余额 
    data=d.loc[:,['用户标识','银行标识','信用卡额度','本期账单余额']].groupby(["用户标识","银行标识"],as_index=False).max()
    gb=data.loc[:,['用户标识','信用卡额度','本期账单余额']].groupby(["用户标识"],as_index=False)
    去重后信用卡额度sum=gb.sum()
    去重后信用卡额度max=gb.max()
    去重后信用卡额度min=gb.min()
    去重后信用卡额度mean=gb.mean()
    去重后信用卡额度std=gb.std()
    去重后信用卡额度var=gb.var()
    去重后信用卡额度sum.columns = ['用户标识','去重后整体信用卡额度sum','去重后本期账单余额sum']
    去重后信用卡额度max.columns = ['用户标识','去重后整体信用卡额度max','去重后本期账单余额max']
    去重后信用卡额度min.columns = ['用户标识','去重后整体信用卡额度min','去重后本期账单余额min']
    去重后信用卡额度mean.columns = ['用户标识','去重后整体信用卡额度mean','去重后本期账单余额mean']
    去重后信用卡额度std.columns = ['用户标识','去重后整体信用卡额度std','去重后本期账单余额std']
    去重后信用卡额度var.columns = ['用户标识','去重后整体信用卡额度var','去重后本期账单余额var']

    feature1=pd.merge(去重后信用卡额度sum, 去重后信用卡额度max,how='outer', on = "用户标识")
    feature1=pd.merge(feature1, 去重后信用卡额度min,how='left', on = "用户标识")
    feature1=pd.merge(feature1, 去重后信用卡额度mean,how='left', on = "用户标识")
    feature1=pd.merge(feature1, 去重后信用卡额度std,how='left', on = "用户标识")
    feature1=pd.merge(feature1, 去重后信用卡额度var,how='left', on = "用户标识")

    bill_feature=pd.merge(feature, feature1,how='left', on = "用户标识")
    return (bill_feature)

In [11]:
credit_train_feature=get_credit_feature(credit_train)
credit_test_feature=get_credit_feature(credit_test)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:946: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  'representation', UserWarning)


##  用户行为表

In [12]:
'''
step1 读入数据 提取时间特征
'''
behaviors_test = pd.read_csv('test/test_behaviors.csv')
behaviors_train = pd.read_csv('train/train_behaviors.csv')

In [13]:
 '''
 step2 特征提取
 '''
def get_behavior_feature(behavior):
    
    behavior['几月']=behavior['日期'].apply(lambda x:re.match(r'(\d+)',x).group())
    behavior['几日']=behavior['日期'].apply(lambda x:re.search(r'\d+\Z',x).group())

    behavior=behavior.drop('日期',axis=1)  #删除原有的日期字段
    behavior=behavior.rename(columns={'星期几':'week','几月':'month','几日':'day'}) #字段重命名

    behavior['month']=behavior['month'].map(lambda x:int(x))
    behavior['day']=behavior['day'].map(lambda x:int(x))

    gb=behavior.groupby(["用户标识"],as_index=False)

    x1=gb['子类型1'].agg({'次数' : 'count','行为1 sum' : 'sum','行为1 max' : 'max','行为1 mean' : 'mean',
                         '行为1 min' : 'min','行为1 std' : 'std','行为1 var' : 'var'})
    x2=gb['子类型2'].agg({'行为2 sum' : 'sum','行为2 max' : 'max','行为2 mean' : 'mean',
                         '行为2 min' : 'min','行为2 std' : 'std','行为2 var' : 'var'})
    # 提取各特征出现的种数
    xx1=gb['子类型1'].apply(lambda x:np.unique(x).size)
    xx2=gb['month'].apply(lambda x:np.unique(x).size)
    xx3=gb['week'].apply(lambda x:np.unique(x).size)
    xx4=gb['day'].apply(lambda x:np.unique(x).size)
    xx5=gb['子类型2'].apply(lambda x:np.unique(x).size)

    x1['子类型1种类']=xx1
    x1['month种类']=xx2
    x1['week种类']=xx3
    x1['day种类']=xx4
    x1['子类型2种类']=xx5

    #各子行为编号下的记录数
    f=behavior
    gf=f['子类型1'].groupby([f['用户标识'],f['行为类型']]).count()
    gf=pd.DataFrame(gf.unstack())
    name=[]
    for i in range(1,9,1):
        name.append('num_行为类型_'+str(i))
    gf.columns=name
    #合并特征
    feature=pd.merge(x1,x2,how='outer',on='用户标识')
    feature=pd.merge(feature,gf,how='outer',on='用户标识')

#-------------------------------------时间纬度下的特征提取------------------------------------
    用户行为=behavior
    #month
    行为数据_max =用户行为['子类型1'].groupby([用户行为['用户标识'],用户行为['month']]).max()
    行为数据_max = pd.DataFrame(行为数据_max.unstack())

    行为数据_min =用户行为['子类型1'].groupby([用户行为['用户标识'],用户行为['month']]).min()
    行为数据_min = pd.DataFrame(行为数据_min.unstack())

    行为数据_count =用户行为['子类型1'].groupby([用户行为['用户标识'],用户行为['month']]).count()
    行为数据_count = pd.DataFrame(行为数据_count.unstack())

    行为_month_整体=pd.merge(行为数据_max,行为数据_min,how="left",on="用户标识")
    行为_month_整体=pd.merge(行为_month_整体,行为数据_count,how="outer",on="用户标识")

    #week
    行为数据_max =用户行为['子类型1'].groupby([用户行为['用户标识'],用户行为['week']]).max()
    行为数据_max = pd.DataFrame(行为数据_max.unstack())

    行为数据_min =用户行为['子类型1'].groupby([用户行为['用户标识'],用户行为['week']]).min()
    行为数据_min = pd.DataFrame(行为数据_min.unstack())

    行为数据_count =用户行为['子类型1'].groupby([用户行为['用户标识'],用户行为['week']]).count()
    行为数据_count = pd.DataFrame(行为数据_count.unstack())

    行为_week_整体=pd.merge(行为数据_max,行为数据_min,how="left",on="用户标识")
    行为_week_整体=pd.merge(行为_week_整体,行为数据_count,how="outer",on="用户标识")

    #day
    行为数据_max =用户行为['子类型1'].groupby([用户行为['用户标识'],用户行为['day']]).max()
    行为数据_max = pd.DataFrame(行为数据_max.unstack())

    行为数据_min =用户行为['子类型1'].groupby([用户行为['用户标识'],用户行为['day']]).min()
    行为数据_min = pd.DataFrame(行为数据_min.unstack())

    行为数据_count =用户行为['子类型1'].groupby([用户行为['用户标识'],用户行为['day']]).count()
    行为数据_count = pd.DataFrame(行为数据_count.unstack())

    行为_day_整体=pd.merge(行为数据_max,行为数据_min,how="left",on="用户标识")
    行为_day_整体=pd.merge(行为_day_整体,行为数据_count,how="outer",on="用户标识")

    #合并
    behavior_feature=pd.merge(feature,行为_month_整体,on='用户标识',how='outer')
    behavior_feature=pd.merge(behavior_feature,行为_week_整体,on='用户标识',how='outer')
    behavior_feature=pd.merge(behavior_feature,行为_day_整体,on='用户标识',how='outer')
    return (behavior_feature)



In [14]:
behaviors_test_feature = get_behavior_feature(behaviors_test)
behaviors_train_feature = get_behavior_feature(behaviors_train)

##  是否逾期表

In [15]:
# 读入数据
train_label = pd.read_csv ('train/train_label.csv')
#test_label_A = pd.read_csv ('test/test_label_A.csv')
#test_label_B = pd.read_csv ('train/train_label_B.csv')
#test_label pd.concat([test_label_A,test_label_B],axis=1)

In [16]:
#---------------------------------特征合并------------------------------------------
#训练集
train_all_feature = pd.merge(profile_train_feature,bank_train_feature,on='用户标识',how='outer')
train_all_feature = pd.merge(train_all_feature,credit_train_feature,on='用户标识',how='outer')
train_all_feature = pd.merge(train_all_feature,behaviors_train_feature,on='用户标识',how='outer')
train_all_feature = pd.merge(train_all_feature,train_label,on='用户标识',how='outer')
train_all_feature

,用户标识,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7,8,...,23_y,24_y,25_y,26_y,27_y,28_y,29_y,30_y,31_y,标签
0,0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,80.0,NaN,NaN,NaN,NaN,120.0,NaN,0
1,1,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,147.0,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.0,NaN,0
3,3,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,5,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,7,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,8,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,9,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,101.0,NaN,NaN,NaN,NaN,NaN,87.0,NaN,1
8,10,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,11,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,248.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [17]:
test_all_feature = pd.merge(profile_test_feature,bank_test_feature,on='用户标识',how='outer')
test_all_feature = pd.merge(test_all_feature,credit_test_feature,on='用户标识',how='outer')
test_all_feature = pd.merge(test_all_feature,behaviors_test_feature,on='用户标识',how='outer')
#all_feature = pd.merge(test_all_feature,test_label,on='用户标识',how='outer')
test_all_feature 

,用户标识,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7,8,...,22_y,23_y,24_y,25_y,26_y,27_y,28_y,29_y,30_y,31_y
0,65407,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,192.0,NaN,NaN,NaN,160.0,NaN,NaN,NaN,112.0,NaN
1,60649,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,36.0,NaN,NaN,NaN,NaN,NaN,NaN
2,40934,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,144.0,NaN,NaN,NaN,NaN,NaN,NaN,189.0
3,1736,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,70821,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,24945,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,164.0,NaN,NaN,NaN,159.0,NaN
6,27297,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,112.0,112.0,NaN,NaN,NaN
7,39013,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0
8,35967,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN
9,71979,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# LightGBM

In [18]:
'''
划分测试集、训练集
'''
train=train_all_feature

train_xy,offline_test = train_test_split(train, test_size = 0.2,random_state=21)
train,val = train_test_split(train_xy, test_size = 0.2,random_state=21)

y = train.标签
X = train.drop(['标签','用户标识'],axis=1)

val_y = val.标签
val_X = val.drop(['标签','用户标识'],axis=1)

offline_test_X=offline_test.drop(['标签','用户标识'],axis=1)

In [19]:
'''
定义评分算法
'''
from sklearn import metrics
def ks(y_predicted, y_true):
    label=y_true
    fpr,tpr,thres = metrics.roc_curve(label,y_predicted,pos_label=1)
    return 'ks',abs(fpr - tpr).max()

## LightGBM训练

In [20]:
'''
 网格搜索，参数优化
'''
from sklearn.model_selection import GridSearchCV
print('Start training...')

estimator = lgb.LGBMRegressor(num_leaves=31)

param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [20, 40]
}
gbm = GridSearchCV(estimator, param_grid)
gbm.fit(X, y)
print('Best parameters found by grid search are:', gbm.best_params_)

Start training...


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best parameters found by grid search are: {'learning_rate': 0.1, 'n_estimators': 40}


In [23]:
'''
训练模型
'''
start_time = time.time()
# create dataset for lightgbm
lgb_train = lgb.Dataset(X, y, free_raw_data=False)
lgb_eval = lgb.Dataset(val_X, val_y, reference=lgb_train,free_raw_data=False)

seed=13
params2 = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    #'metric': 'binary_logloss',
    'metric': 'auc',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.7,
    'bagging_fraction': 1,
    'bagging_freq': 10,
    'verbose': 0,
    #'num_iterations':500,
    'tree_learner':'serial',  
    'min_data_in_leaf':10,
    'feature_fraction_seed':seed,
    'bagging_seed':seed,
    'metric_freq':1 
}

print('Start training...')
# train
gbm = lgb.train(params2,
                lgb_train,
                num_boost_round=5000,
                valid_sets=lgb_eval,
                early_stopping_rounds=500)
#交叉验证训练模型
#num_round = 10
#gbm = lgb.cv(params2, lgb_train, num_boost_round=5000, nfold=5)

#print('Save model...')
# save model to file
#gbm.save_model('20190818_A.model') # 用于存储训练出的模型

Start training...
[1]	valid_0's auc: 0.717091
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's auc: 0.728821
[3]	valid_0's auc: 0.731926
[4]	valid_0's auc: 0.735494
[5]	valid_0's auc: 0.738024
[6]	valid_0's auc: 0.740404
[7]	valid_0's auc: 0.744094
[8]	valid_0's auc: 0.747601
[9]	valid_0's auc: 0.750698
[10]	valid_0's auc: 0.752538
[11]	valid_0's auc: 0.753342
[12]	valid_0's auc: 0.754868
[13]	valid_0's auc: 0.755667
[14]	valid_0's auc: 0.757181
[15]	valid_0's auc: 0.757389
[16]	valid_0's auc: 0.757674
[17]	valid_0's auc: 0.758324
[18]	valid_0's auc: 0.758669
[19]	valid_0's auc: 0.759906
[20]	valid_0's auc: 0.760727
[21]	valid_0's auc: 0.761303
[22]	valid_0's auc: 0.761445
[23]	valid_0's auc: 0.762784
[24]	valid_0's auc: 0.763037
[25]	valid_0's auc: 0.762842
[26]	valid_0's auc: 0.763548
[27]	valid_0's auc: 0.763882
[28]	valid_0's auc: 0.763874
[29]	valid_0's auc: 0.763905
[30]	valid_0's auc: 0.764313
[31]	valid_0's auc: 0.764783
[32]	valid_0's auc: 0.764862


[277]	valid_0's auc: 0.766596
[278]	valid_0's auc: 0.766539
[279]	valid_0's auc: 0.766581
[280]	valid_0's auc: 0.766406
[281]	valid_0's auc: 0.766176
[282]	valid_0's auc: 0.766131
[283]	valid_0's auc: 0.766119
[284]	valid_0's auc: 0.766201
[285]	valid_0's auc: 0.76609
[286]	valid_0's auc: 0.766198
[287]	valid_0's auc: 0.76611
[288]	valid_0's auc: 0.766108
[289]	valid_0's auc: 0.766004
[290]	valid_0's auc: 0.765909
[291]	valid_0's auc: 0.766039
[292]	valid_0's auc: 0.766071
[293]	valid_0's auc: 0.765886
[294]	valid_0's auc: 0.765808
[295]	valid_0's auc: 0.765762
[296]	valid_0's auc: 0.76571
[297]	valid_0's auc: 0.765805
[298]	valid_0's auc: 0.765632
[299]	valid_0's auc: 0.765604
[300]	valid_0's auc: 0.765605
[301]	valid_0's auc: 0.765706
[302]	valid_0's auc: 0.765651
[303]	valid_0's auc: 0.765538
[304]	valid_0's auc: 0.765659
[305]	valid_0's auc: 0.765601
[306]	valid_0's auc: 0.765569
[307]	valid_0's auc: 0.765619
[308]	valid_0's auc: 0.765479
[309]	valid_0's auc: 0.765616
[310]	valid_0

[552]	valid_0's auc: 0.761235
[553]	valid_0's auc: 0.761078
[554]	valid_0's auc: 0.761069
[555]	valid_0's auc: 0.761098
[556]	valid_0's auc: 0.761131
[557]	valid_0's auc: 0.761114
[558]	valid_0's auc: 0.761032
[559]	valid_0's auc: 0.760859
[560]	valid_0's auc: 0.760949
[561]	valid_0's auc: 0.7608
[562]	valid_0's auc: 0.760812
[563]	valid_0's auc: 0.760756
[564]	valid_0's auc: 0.760598
[565]	valid_0's auc: 0.760545
[566]	valid_0's auc: 0.760754
[567]	valid_0's auc: 0.760738
[568]	valid_0's auc: 0.760711
[569]	valid_0's auc: 0.760693
[570]	valid_0's auc: 0.76067
[571]	valid_0's auc: 0.760584
[572]	valid_0's auc: 0.760479
[573]	valid_0's auc: 0.760391
[574]	valid_0's auc: 0.760314
[575]	valid_0's auc: 0.760307
[576]	valid_0's auc: 0.760429
[577]	valid_0's auc: 0.760389
[578]	valid_0's auc: 0.760254
[579]	valid_0's auc: 0.760074
[580]	valid_0's auc: 0.760099
[581]	valid_0's auc: 0.760186
[582]	valid_0's auc: 0.760241
[583]	valid_0's auc: 0.760084
[584]	valid_0's auc: 0.760069
[585]	valid_0

##  LightGBM预测

In [24]:
print ("跑到这里了model.predict")
preds_offline = gbm.predict(offline_test_X, num_iteration=gbm.best_iteration)
offline = offline_test[['用户标识','标签']]
offline['预测'] = preds_offline

#预测测试集
test_pre = gbm.predict(test_all_feature, num_iteration=gbm.best_iteration) 
test_predict = test_all_feature[['用户标识']]
test_predict ['预测'] = test_pre

#模型评价
from sklearn.metrics import roc_auc_score
auc_score = roc_auc_score(offline.标签,offline.预测)
print('AUC：')
print(auc_score)

print("KS;")
print(ks(offline.预测,offline.标签))


跑到这里了model.predict


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


AUC：
0.7620273537012541
KS;
('ks', 0.39617228835978835)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [25]:
'''
计算特征得分
'''
df = pd.DataFrame(X.columns.tolist(), columns=['feature'])
df['importance']=list(gbm.feature_importance())
df=df.sort_values(["importance"],ascending=False)
df

,feature,importance
540,整体上期还款金额var与整体上期账单金额var差值,90
582,num_行为类型_7,88
526,整体上期还款金额median与整体上期账单金额median差值,60
581,num_行为类型_6,60
505,整体上期还款金额max与整体上期账单金额max差值,55
515,整体上期还款金额mean,51
578,num_行为类型_3,50
580,num_行为类型_5,48
536,整体上期还款金额var,48
637,3_y_x,44


In [26]:
test_predict

,用户标识,预测
0,65407,0.699995
1,60649,0.884669
2,40934,0.634576
3,1736,0.597039
4,70821,0.839015
5,24945,0.670621
6,27297,0.748343
7,39013,0.822442
8,35967,0.712896
9,71979,0.723551
